
### Ipopt, cyipopt demo
**Jonathan Cangelosi, Matthias Heinkenschloss**,
**Department of Computational Applied Mathematics and Operations Research**, **Rice University**.

Last modified April 23, 2025

Adopted from https://cyipopt.readthedocs.io/en/stable/tutorial.html#scipy-compatible-interface, https://cyipopt.readthedocs.io/en/stable/tutorial.html#problem-interface

Consider NLP
\begin{align*}   
   \text{minimize } &  f(x),  \\
   \text{subject to }  & g_{\min} \le g(x) \le  g_{\max},\\
                     & x_{\min} \le x \le  x_{\max},
\end{align*}
where $f: \mathbb{R}^n \rightarrow \mathbb{R}$ and
$g: \mathbb{R}^n \rightarrow \mathbb{R}^m$ are smooth functions.

The Lagrangian associated with this NLP is
\begin{align*}  
      L(x,\lambda) = f(x) + \lambda^T g(x).
\end{align*}  

Ipopt requires evaluations of $f$, $g$, the gradient $\nabla f(x) \in \mathbb{R}^n$, 
the Jacobian $g'(x) \in \mathbb{R}^{m \times n}$, and the Hessian of the Lagrangian
with respect to $x$, $\nabla_x^2 L(x,\lambda)$.

We use jax to compute the objective function gradient, the constraint Jacobian, and the Hessian of the Lagrangian.  For more details on Jax see
https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html


In [50]:
import jax
# Enable 64 bit floating point precision
jax.config.update("jax_enable_x64", True)
 # We use the CPU instead of GPU und mute all warnings if no GPU/TPU is found.
jax.config.update("jax_platform_name", "cpu")
import jax.numpy as jnp
from jax import jit, grad, jacfwd, jacrev

import cyipopt
import numpy as np



Example problem HS71
\begin{align*}   
   \text{minimize } \;&  x_1 x_4 \Big( x_1 + x_2 + x_3 \big) + x_3,  \\
   \text{subject to }\;  & x_1 x_2 x_3 x_4 \ge 25,\\
                     & x_1^2 +  x_2^2 +  x_3^2 +  x_4^2 = 40,\\
                     & 1 \le x \le  5
\end{align*}
from the standard Hock-Schittkowski collection.
Initial iterate $x_0 = (1, 5, 5, 1)$.

In [51]:
class HS071():
    def __init__(self):
        self.xmin = [1.0, 1.0, 1.0, 1.0]  # lower x-bound
        self.xmax = [5.0, 5.0, 5.0, 5.0]  # upper x-bound
        self.gmin = [25.0, 40.0]          # lower g-bound
        self.gmax = [2.0e19, 40.0]        # upper g-bound
        self.x0 = [1.0, 5.0, 5.0, 1.0]    # initial iterate

    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        return x[0] * x[3] * jnp.sum(x[0:3]) + x[2]

    def constraints(self, x):
        """Returns the constraints."""
        return jnp.array((jnp.prod(x), jnp.dot(x, x)))
    
    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return grad(self.objective)(x)

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return jacrev(self.constraints)(x)

    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.

        return np.nonzero(np.tril(np.ones((4, 4))))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""

        H = obj_factor*jacfwd(jacrev(self.objective))(x)
        H += jacfwd(lambda x: jnp.matmul(lagrange, jacrev(self.constraints)(x)))(x)

        row, col = self.hessianstructure()

        return H[row, col]

    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""
        print('iter  obj_value  inf_pr     inf_du     mu        '\
              'd_norm     reg_size   alpha_du   alpha_pr    ls_trials')
        print('{0:4d} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e} '\
                '{6:10.3e} {7:10.3e} {8:10.3e} {9:4d}'.format(\
              iter_count, obj_value, inf_pr, inf_du, mu,\
              d_norm, regularization_size, alpha_du, alpha_pr, ls_trials))

In [52]:
nlp = cyipopt.Problem(
   n=len(HS071().x0),
   m=len(HS071().gmin),
   problem_obj=HS071(),
   lb=HS071().xmin,
   ub=HS071().xmax,
   cl=HS071().gmin,
   cu=HS071().gmax,
)

In [53]:
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-7)

In [54]:
x, info = nlp.solve(HS071().x0)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter  obj_value  inf_pr     inf_du     mu        d_norm     reg_size   alpha_du   alpha_pr    ls_trials
   0  1.611e+01  1.124e+01  5.276e-01  1.000e+00  0.00

In [55]:
x

array([0.99999999, 4.74299964, 3.82114998, 1.37940829])

In [56]:
info

{'x': array([0.99999999, 4.74299964, 3.82114998, 1.37940829]),
 'g': array([24.99999975, 40.        ]),
 'obj_val': 17.014017140224134,
 'mult_g': array([-0.55229366,  0.16146856]),
 'mult_x_L': array([1.08787121e+00, 2.67160763e-12, 3.54491149e-12, 2.63544924e-11]),
 'mult_x_U': array([2.49994270e-12, 3.89698412e-11, 8.48203576e-12, 2.76216310e-12]),
 'status': 0,
 'status_msg': b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'}